## Notebook for working with shapely files

### Opened Tuesday 12-13

#### Couldn't create geopanda df correctly when I imported from a csv so the first few cells are all of the transformations from the clean_parse notebook that created a working object

In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import os
from area import area

file_path = ('data\kml\My Places_dec_9_22.kml')

with open(file_path) as file:

    xml_data = file.read()

# Initializing soup variables
soup = BeautifulSoup(xml_data, 'xml')

folders = soup.Document.Folder
list = soup.Document.Folder.find_all('Folder')
layers = soup.Document.Folder.Folder
polygons = soup.Document.Folder.Placemark.Polygon

## Create a dataframe to hold the data parsed from xml

df = pd.DataFrame(columns=['field', 'foul', 'fop'])



## Loop through the folders and extract the data


    
i = 0   

for i in range(len(list)):

    folders = list[i]
    field_name = folders.find('name').text
    foul = folders.find_all('coordinates')[0].text
    fop = folders.find_all('coordinates')[1].text

    row = {
        'field': field_name,
        'foul': foul,
        'fop': fop
    }

    i+=1

    df = df.append(row, ignore_index=True)

## Clean up the data

# remove new line and and space characters from coordinates
df = df.replace(r'\n','', regex=True) 
df = df.replace(r'\t','', regex=True) 


## Drop any duplicate rows
df = df.drop_duplicates(subset=['field'], keep='first')

## Drop and rows with empty fields
df = df[(df != 0).all(1)]


## remove numberic characters and . from field names
df['field'] = df['field'].str.replace(r'\d+', '')
df['field'] = df['field'].str.replace(r'\.', '')
    

C:\Users\Justin\AppData\Local\Temp\ipykernel_5444\3218802594.py:48: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row, ignore_index=True)
C:\Users\Justin\AppData\Local\Temp\ipykernel_5444\3218802594.py:48: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row, ignore_index=True)
C:\Users\Justin\AppData\Local\Temp\ipykernel_5444\3218802594.py:48: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row, ignore_index=True)
C:\Users\Justin\AppData\Local\Temp\ipykernel_5444\3218802594.py:48: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row, ignore_index=True)
C:\Users\Justin\AppData\Local\Te

In [3]:

## Create a (lat, long) pair for home plate 
## Just grab first uple form the foul ground coordinates

df['home_plate'] = df['foul'].str.split(' ').str[0]

## Calculate the area of each polygon in square meters
df['foul_area'] = df['foul'].apply(lambda x: area({'type': 'Polygon', 'coordinates': [[tuple(map(float, coord.split(','))) for coord in x.split()]]}))
df['fop_area'] = df['fop'].apply(lambda x: area({'type': 'Polygon', 'coordinates': [[tuple(map(float, coord.split(','))) for coord in x.split()]]}))

# Convert the area to square feet
df['foul_ft'] = df['foul_area'].apply(lambda x: x*10.7639)
df['fop_ft'] = df['fop_area'].apply(lambda x: x*10.7639)

# drop the area columns
df = df.drop(['foul_area', 'fop_area'], axis=1)

# calculate the ratio of foul ground to total area
df['foul_pct'] = (df['foul_ft']/(df['fop_ft']+df['foul_ft']))*100

## Convert the coordinates to a list of tuples
df['foul_poly'] = df['foul'].apply(lambda x: [tuple(map(float, coord.split(','))) for coord in x.split()])
df['fop_poly'] = df['fop'].apply(lambda x: [tuple(map(float, coord.split(','))) for coord in x.split()])


## Create shapely polygons from the coordinates

from shapely.geometry import Polygon
import geopandas as gpd

# df['foul'] = df['foul'].apply(lambda x: Polygon(x))
# df['fop'] = df['fop'].apply(lambda x: Polygon(x))


df.head()


,field,foul,fop,home_plate,foul_ft,fop_ft,foul_pct,foul_poly,fop_poly
0,Adrian College,"-84.0697145,41.901861,0 -84.0703958,41.9026485...","-84.0697145,41.901861,0 -84.0687248,41.9023461...","-84.0697145,41.901861,0",14280.359302,106000.287111,11.872533,"[(-84.0697145, 41.901861, 0.0), (-84.0703958, ...","[(-84.0697145, 41.901861, 0.0), (-84.0687248, ..."
1,Airport High School - Carleton,"-83.37652180000001,42.0373919,0 -83.3765245,42...","-83.37652180000001,42.0373919,0 -83.3753591,42...","-83.37652180000001,42.0373919,0",21261.726079,88074.049869,19.446266,"[(-83.3765218, 42.0373919, 0.0), (-83.3765245,...","[(-83.3765218, 42.0373919, 0.0), (-83.3753591,..."
2,Aldai Stevenson High School,"-83.01445630000001,42.5871203,0 -83.0156645999...","-83.01445630000001,42.5871203,0 -83.014231,42....","-83.01445630000001,42.5871203,0",20786.331595,88749.278963,18.976780,"[(-83.0144563, 42.5871203, 0.0), (-83.0156646,...","[(-83.0144563, 42.5871203, 0.0), (-83.014231, ..."
4,Algonac High School,"-82.58239759999999,42.6286202,0 -82.5813153999...","-82.58239759999999,42.6286202,0 -82.5826256,42...","-82.58239759999999,42.6286202,0",30426.885427,83361.733170,26.739832,"[(-82.5823976, 42.6286202, 0.0), (-82.5813154,...","[(-82.5823976, 42.6286202, 0.0), (-82.5826256,..."
5,Allen Park High School,"-83.2273711,42.2455509,0 -83.2285244,42.245525...","-83.2273711,42.2455509,0 -83.22739919999999,42...","-83.2273711,42.2455509,0",21933.085958,94654.973990,18.812463,"[(-83.2273711, 42.2455509, 0.0), (-83.2285244,...","[(-83.2273711, 42.2455509, 0.0), (-83.2273992,..."


## end of code from clean_notebook_parse

### returning usualble gdf

In [ ]:
# df.info()
# from shapely.geometry import Point

# ## Convert the home plate coordinate into a shapely point
# df['home_pt'] = df['home_plate'].apply(lambda x: Point(tuple(map(float, x.split(',')))))
# df['home_pt'] = df['home_pt'].apply(lambda x: x.buffer(0.0001))


# ## change the foul and fop polygons strings to linestrings,
# df['foul'] = df['foul'].apply(lambda x: x.exterior.coords[:])
# df['fop'] = df['fop'].apply(lambda x: x.exterior.coords[:])

# ## drop the first and last coordinate from the linestrings
# df['foul'] = df['foul'].apply(lambda x: x[1:-1])
# df['fop'] = df['fop'].apply(lambda x: x[1:-1])


# from shapely.geometry import LineString

# # ## make a line out of the fop coordinates
# # df['fop_line'] = df['fop'].apply(lambda x: LineString(x))

# df.head()



##### WORKING TO HERE

df.head()

### I think I have a usable linestring to measure to.

#### for some reason home_pt is getting saved as a polygon, need to figure that out

## below I is for trying to measure the distance


In [7]:
#### Sunday 12/13/20
### going to try a new approach to measue the longest and shortest distance from home plate to the fop_line
### using a buffer around the home plate point and then finding the intersection of the buffer and the fop_line

### using havesine formula to calculate the distance between two points per this link
### https://towardsdatascience.com/calculating-distance-between-two-geolocations-in-python-26ad3afe287b

import haversine as hs

x = 10

loc1 = df['home_plate'][x]
loc2 = df['fop'][x]

hs.haversine(loc1, loc2)


ValueError: too many values to unpack (expected 2)

In [ ]:
### New Block Wednesday Morning
### Caculate the distance from home plate to the fop line using hausdorff distance
##### I think this is on the wrong track - going back to trying with shapely
# from scipy.spatial.distance import directed_hausdorff
# from scipy.spatial.distance import cdist

# x = 10

# # tranform home_plate into a 2d array
# df['home_plate'] = df['home_plate'].apply(lambda x: np.array(tuple(map(float, x.split(',')))))

# # df['home_plate'] = df['home_plate'].apply(lambda x: Point(tuple(map(float, x.split(',')))))

# h_distance = cdist(df['home_plate'][x], df['fop_line'][x], lambda u, v: directed_hausdorff(u, v)[0])

# h_dist_df = pd.DataFrame(h_distance, columns=['hausdorff_dist'])
# h_dist_df.head()

In [ ]:
# import math

# x = 0
# df['home_plate'] = df['home_plate'].apply(lambda x: Point(tuple(map(float, x.split(',')))))
# pt = Point(df['home_plate'][x])
# ls = df['fop_line'][x]

# print(df['field'][x])
# print(pt.distance(ls)* 364519.8349747314)
# print(pt.hausdorff_distance(ls) * 364519.8349747314)

df['home_plate'] = df['home_plate'].apply(lambda x: Point(tuple(map(float, x.split(',')))))

In [ ]:
### Playing with the Hausdorff distance

from shapely.geometry import Point
# from shapely import hausdorff_distance

# df['home_plate'] = df['home_plate'].apply(lambda x: Point(tuple(map(float, x.split(',')))))

x = 0



# pt = df['home_plate'][x]
# ls = df['fop_line'][x]
# h_dist = pt.hausdorff_distance(ls) * 328084
# h_dist_2 = ls.hausdorff_distance(pt) * 328084
# print(h_dist)
# print(h_dist_2)


x = 0

for x in range(len(df['home_plate'])):
    try:
        pt = df['home_plate'][x]
        ls = df['fop_line'][x]
        h_dist = pt.hausdorff_distance(ls) * 328084
        df['h_dist'][x] = h_dist
        x+=1
    except KeyError:
        pass


df.head()

# gdf_line = df['fop_line']

# gdf_hp = df['home_pt']

# df['h_dist'] = hausdorff_distance(gdf_hp, gdf_line, densify=0.5) * 328084

# print(df['h_dist'])

# for x in range(len(df['home_plate'])):
#     try:
#         pt = df['home_plate'][x]
#         ls = df['fop_line'][x]
#         df['h_dist'] = pt.hausdorff_distance(ls) * 328084
#         x+=1
#     except KeyError:
#         pass
# return(df)


    

# df.head()

    
    
# print(df['field'][x])
# # print(pt.distance(ls)* 364519.8349747314)
# print(pt.hausdorff_distance(ls) * 328084)
# print('')

In [ ]:
#### TRIED TO USE THE MIN FUNCTION TO GET MAX BUT IT CAN'T BE DONE THIS WAY

# #### trying to work with a function i found from stackexchange

# # https://gis.stackexchange.com/questions/376696/geopandas-distance-optimisation

# import geopandas as gpd
# from shapely.geometry import Point, LineString
# import pandas as pd
# import random

# gdf = gpd.GeoDataFrame(geometry=df['fop_line'], crs="EPSG:4326")
# gdf2 = gpd.GeoDataFrame(geometry=df['home_plate'], crs="EPSG:4326")

# def get_max_distance(left, right, initial_buffer):
#     """get distance from right to left"""
#     buffered = right.buffer(initial_buffer)
#     b = initial_buffer

#     distances_max = []
#     for i in range(len(buffered)):
#         geom = buffered.geometry.iloc[i]
#         query = right.sindex.query(geom)
#         while query.size == 0:
#             query = right.sindex.query(geom.buffer(b))
#             b += initial_buffer
#         distances_max.append(right.iloc[query].hausdorff_distance(left.geometry.iloc[i]))

#     return pd.Series(distances_max, index=left.index)

# gdf['distance_to_x_max'] = get_max_distance(gdf, gdf2, 50)

# gdf['dist_in_meters_max'] = gdf['distance_to_x_max'] * 111000 * 3.28084
# gdf['dist_in_feet_max'] = gdf['dist_in_meters_max'] * 3.28084

# df['max_dist'] = gdf['dist_in_feet_max']

# df.head()

In [ ]:
import geopandas as gpd
import shapely
from shapely.geometry import Point, LineString
import pandas as pd
import random

gdf = gpd.GeoDataFrame(geometry=df['fop_line']).set_crs('EPSG:3857')#.to_crs('EPSG:3857')

gdf2 = gpd.GeoDataFrame(geometry=df['home_plate']).set_crs('EPSG:3857')#.to_crs('EPSG:3857')

def get_nearest_distance(left, right, initial_buffer):
    """get distance from left to right"""
    buffered = left.buffer(initial_buffer)
    b = initial_buffer

    distances = []
    for i in range(len(buffered)):
        geom = buffered.geometry.iloc[i]
        query = right.sindex.query(geom)
        while query.size == 0:
            query = right.sindex.query(geom.buffer(b))
            b += initial_buffer
        distances.append(right.iloc[query].distance(left.geometry.iloc[i]).min())

    return pd.Series(distances, index=left.index)

gdf['distance_to_x'] = get_nearest_distance(gdf, gdf2, 50)

# convert to feet

gdf['dist_in_meters'] = (gdf['distance_to_x']  * 111000 *3.28084)
gdf['dist_in_feet'] = gdf['dist_in_meters']

df['min_dist'] = gdf['dist_in_feet']

df.head()

In [ ]:
df.describe()

In [ ]:


## Export the data to a csv file to check the results

export_df = df.filter(['field', 'foul_ft', 'fop_ft', 'foul_pct', 'dist_calc'], axis=1)
export_df.to_csv('dist_calc.csv', index=False)

#### Output Looks Pretty Good!!!
### values are right around where i would expect them to be



In [ ]:
#### 12-14-.....1 am
### Don't really understand what this method is returning
### average distance maybe?
### avg distance would be good to know but what I am really interested in is the max and min distances



import shapely.wkt
import geopandas as gpd

gdf_line = gpd.GeoDataFrame(geometry=df['fop_line'], crs="EPSG:4326")

gdf_hp = gpd.GeoDataFrame(geometry=df['home_pt'], crs="EPSG:4326")
utm = gdf_line.estimate_utm_crs()

for x in gdf_line:
    gdf_line.to_crs(utm)
    df['avg_dist'] = (gdf_line.to_crs(utm).distance(gdf_hp.to_crs(utm)))*3.28084
    df['max_dist'] = (gdf_line.to_crs(utm).distance(gdf_hp.to_crs(utm))


In [ ]:
df.head()

In [ ]:
from pyproj import Geod

geod = Geod(ellps="WGS84")

f"{geod.geometry_length(df['fop_line'][10])} meters"
f"{geod.geometry_length(df['fop_line'][10])*3.28084} feet"

## calculate the distance from home plate to the fop line
df['dist_to_fop'] = df['home_pt'].apply(lambda x: geod.geo(df['fop_line'][10]))

df.head()


In [ ]:




## Calculate the distance from home plate to the outfield line
df['home_pt'] = Point(df['home_plate'])
df['dist'] = df.apply(lambda x: x['home_pt'].distance(x['fop_line']), axis=1)
df.head()

In [ ]:
### Import folium and create map
x = 40

home_pt = Point(df['home_pt'][x])

foul_poly = df['foul'][x]
fop_poly = df['fop'][x]

foul_geom = Polygon(foul_poly)
fop_geom = Polygon(fop_poly)
polygon = gpd.GeoDataFrame(index=[x], crs='epsg:3857', geometry=[fop_geom])       


print(polygon.boundary.distance(home_pt))

test_pt = polygon.exterior.interpolate(polygon.exterior.project(home_pt))

# print(test_pt)
# print(home_pt)
# print(test_pt.distance(home_pt))

# 1 METER = 3.28084 FEET


# import folium
# m = folium.Map(home, zoom_start=5, tiles='cartodbpositron')
# folium.GeoJson(polygon).add_to(m)
# folium.LatLngPopup().add_to(m)
# m

In [ ]:
x = 12

home = [df['home_lat'][x], df['home_long'][x]]

foul_poly = df['foul'][x]
fop_poly = df['fop'][x]

foul_geom = Polygon(foul_poly)
fop_geom = Polygon(fop_poly)

df['foul'].boundary.distance(home)

In [ ]:
# https://gis.stackexchange.com/questions/294206/how-to-create-a-simple-polygon-from-coordinates-in-geopandas-with-python

# from shapely import wkt

# import geopandas as gpd
# from shapely.geometry import Polygon

# lon_lat_list = df['foul'][3]
# polygon_geom = Polygon(lon_lat_list)
# polygon = gpd.GeoDataFrame(index=[0], crs='epsg:4326', geometry=[polygon_geom])       

# import folium
# m = folium.Map([50.854457, 4.377184], zoom_start=5, tiles='cartodbpositron')
# folium.GeoJson(polygon).add_to(m)
# folium.LatLngPopup().add_to(m)
# m
# df['foul_shapely'] = df['foul_str'].apply(lambda x: wkt.loads(x)) # only works with strings
# df.head()